In [65]:
import requests
import sqlite3
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from datetime import datetime
os.getcwd()

'c:\\Users\\color\\Downloads'

In [106]:
def log_progress(message):
    with open('code_log.txt', 'a') as f:
        f.write(str(datetime.now()) + ' : ' + message + '\n')
        f.write('\n')
        f.close()
        print(str(datetime.now()) + ' : ' + message + '\n')

In [74]:
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    table = data.find_all('tbody')[0].find_all('tr')[1:]
    df = pd.DataFrame(columns=table_attribs)
    for row in table:
        df.loc[len(df.index)] = [row.findAll('td')[1].text.strip(), row.findAll('td')[2].text.strip()]  

    log_progress('Data extraction complete. Initiating Transformation process')
    print(df)
    return df

In [94]:
def transform(df, csv_path):
    exchange_rate = {}
    for index, row in pd.read_csv(csv_path).iterrows():
        exchange_rate[row[0]] = float(row[1])
    df['MC_GBP_Billion'] = [np.round(float(x)*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(float(x)*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(float(x)*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]
    log_progress('Data transformation complete. Initiating Loading process')
    print(df)
    return df

In [95]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)
    log_progress('Data saved to CSV file')

In [108]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    log_progress('Data loaded to Database as a table, Executing queries')

In [111]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)
    log_progress(f'Query -- {query_statement} -- Process Complete')

In [112]:
if os.path.exists('code_log.txt'):
    os.remove('code_log.txt')
    
log_progress('Preliminaries complete. Initiating ETL process')

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
table_attribs = ["Name","MC_USD_Billion"]
output_csv_path = 'Largest_banks_data.csv'
log_file = 'code_log.txt'

df = extract(url, table_attribs)
df = transform(df, csv_path)
load_to_csv(df, output_csv_path)

sql_connection = sqlite3.connect(db_name)
load_to_db(df, sql_connection, table_name)

query = ['SELECT * FROM Largest_banks', 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks', 'SELECT Name from Largest_banks LIMIT 5']
for i in query:
    run_query(i, sql_connection)

sql_connection.close()

2024-03-18 23:42:20.348864 : Preliminaries complete. Initiating ETL process

2024-03-18 23:42:21.179585 : Data extraction complete. Initiating Transformation process

                                      Name MC_USD_Billion
0                           JPMorgan Chase         432.92
1                          Bank of America         231.52
2  Industrial and Commercial Bank of China         194.56
3               Agricultural Bank of China         160.68
4                                HDFC Bank         157.91
5                              Wells Fargo         155.87
6                        HSBC Holdings PLC         148.90
7                           Morgan Stanley         140.83
8                  China Construction Bank         139.82
9                            Bank of China         136.81
2024-03-18 23:42:22.193489 : Data transformation complete. Initiating Loading process

                                      Name MC_USD_Billion  MC_GBP_Billion  \
0                           JPM